# Building an Advanced RAG System with AI21's Jamba-1.5-large

This notebook demonstrates implementing a Retrieval Augmented Generation (RAG) system using AI21's Jamba-1.5-large language model. Jamba-1.5-large features a 256k token context window, making it highly effective for RAG applications by allowing:

- Processing of larger chunks of retrieved content
- Better handling of long-form context
- More comprehensive document analysis

We'll combine this with vector storage and embeddings to create an efficient information retrieval and generation pipeline.

In [1]:
from langchain_ai21 import ChatAI21

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [5]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [1]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()


api_key=os.getenv("AI21_API_KEY", None)

from langchain_ai21 import ChatAI21

llm =ChatAI21(model="jamba-1.5-large",api_key=api_key)

In [2]:
a=llm.invoke("Give me 10 sentences that only ends in word apple ")

a.content

'Sure, here are 10 sentences that end with the word "apple":\n\n1. The red fruit hanging from the tree is a juicy apple.\n2. She took a bite of the crisp apple.\n3. The teacher gave each student a shiny apple.\n4. He bought a basket of fresh apples from the market apple.\n5. The orchard was filled with rows of apple trees apple.\n6. For dessert, they served a warm apple pie apple.\n7. The apple cider was sweet and refreshing apple.\n8. She made a delicious apple crumble for the party apple.\n9. The grocery store had a sale on Granny Smith apples apple.\n10. He planted an apple seed in his backyard, hoping it would grow into a tree apple.'

In [3]:
from langchain_voyageai import VoyageAIEmbeddings

In [7]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()


api=os.getenv("VOYAGE_API_KEY")


embeddings = VoyageAIEmbeddings(model="voyage-3",api_key=api)




In [8]:
pinecone_api_key=os.getenv("PINECONE_API_KEY")

In [9]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index_name = "my-first"
index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

# Loading Documents into RAG System

We'll use the LangChain PDF loader to extract content from PDF documents. While there are several alternatives available:

## Document Loading Options
- **LangChain PDF Loader(It provides various options like using libraries PYMUPDF,PYPDF,PDFPLUMBER)** (current choice)
- LlamaParser
- AWS Textract
- Azure AI Document Intelligence
- Multimodal LLMs (GPT-4V, Gemini Pro Vision)

The LangChain PDF loader provides a simple and effective way to extract text content while maintaining document structure and metadata.

In [21]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
    "data/eye_eurp.pdf",
)

In [22]:
docs = loader.load()
docs[1]

Document(metadata={'source': 'data/eye_eurp.pdf', 'page': 1, 'page_label': '2'}, page_content="2/61 \nTable of contents \n \nKey Takeaways 3 \nIntroduction - The fundamental guarantees of a digital society 5 \n1. The technical maturity of facial recognition technologies paves the way for their \ndeployment 8 \n1.1. A maturity in line with the dynamics of artificial intelligence technologies 8 \n1.2. The field of facial recognition encompasses a diversity of uses 10 \n1.2.1. Varied uses with different levels of risk 10 \n1.2.2. Overlap with other technologies raises further concerns 14 \n1.3. Facial recognition technologies are not foolproof 15 \n1.3.1. The inherent shortcomings of facial recognition technologies 15 \n1.3.2. A perpetual technological race to correct their negative effects 19 \n1.3.3. Probabilistic technologies prone to human deficiencies 20 \n2. An inconsistent and efficient application of the legal framework 22 \n2.1. Facial recognition technologies are relatively well

In [18]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
all_splits = text_splitter.split_documents(docs)


In [20]:
all_splits

[Document(metadata={'source': 'data/week1.pdf', 'page': 0, 'page_label': '1'}, page_content='Copyright Notice\nThese slides are distributed under the Creative Commons License.\nDeepLearning.AI makes these slides available for educational purposes. You may not use or \ndistribute these slides for commercial purposes. You may make copies of these slides and \nuse or distribute them for educational purposes as long as you cite DeepLearning.AI as the \nsource of the slides.\nFor the rest of the details of the license, see \nhttps://creativecommons.org/licenses/by-sa/2.0/legalcode'),
 Document(metadata={'source': 'data/week1.pdf', 'page': 1, 'page_label': '2'}, page_content='Generative AI & \nLarge Language \nModels (LLMs)\n USE CASES, \nPROJECT LIFECYCLE, AND \nMODEL PRE-TRAINING'),
 Document(metadata={'source': 'data/week1.pdf', 'page': 2, 'page_label': '3'}, page_content='Generative AI & \nLarge Language Model \nUse Cases & Model \nLifecycle'),
 Document(metadata={'source': 'data/week1.p